SQL file generated successfully.


In [5]:
import pandas as pd
import numpy as np
from faker import Faker
from sqlalchemy import create_engine

# Initialize Faker
fake = Faker()

# Generate dummy data for Freelancers
def create_freelancers(n):
    data = {
        'freelancer_id': range(1, n + 1),
        'name': [fake.name() for _ in range(n)],
        'email': [fake.email() for _ in range(n)],
        'phone': [fake.phone_number() for _ in range(n)],
        'skills': [', '.join(fake.words(nb=3)) for _ in range(n)],
        'rating': [round(np.random.uniform(1, 5), 1) for _ in range(n)],
        'hourly_rate': [round(np.random.uniform(20, 100), 2) for _ in range(n)],
        'profile_description': [fake.text(max_nb_chars=200) for _ in range(n)]
    }
    return pd.DataFrame(data)

# Generate dummy data for Clients
def create_clients(n):
    data = {
        'client_id': range(1, n + 1),
        'name': [fake.company() for _ in range(n)],
        'email': [fake.email() for _ in range(n)],
        'phone': [fake.phone_number() for _ in range(n)],
        'company': [fake.company() for _ in range(n)],
        'profile_description': [fake.text(max_nb_chars=200) for _ in range(n)]
    }
    return pd.DataFrame(data)

# Generate dummy data for Proposals
def create_proposals(freelancers_df, clients_df, n):
    data = {
        'proposal_id': range(1, n + 1),
        'freelancer_id': np.random.choice(freelancers_df['freelancer_id'], size=n),
        'client_id': np.random.choice(clients_df['client_id'], size=n),
        'project_title': [fake.sentence(nb_words=5) for _ in range(n)],
        'description': [fake.text(max_nb_chars=500) for _ in range(n)],
        'bid_amount': [round(np.random.uniform(100, 2000), 2) for _ in range(n)],
        'submission_date': [fake.date_between(start_date='-1y', end_date='today').strftime('%Y-%m-%d') for _ in range(n)],
        'status': np.random.choice(['submitted', 'accepted', 'rejected'], size=n)
    }
    return pd.DataFrame(data)

# Generate dummy data for Reviews
def create_reviews(freelancers_df, clients_df, n):
    data = {
        'review_id': range(1, n + 1),
        'freelancer_id': np.random.choice(freelancers_df['freelancer_id'], size=n),
        'client_id': np.random.choice(clients_df['client_id'], size=n),
        'rating': np.random.randint(1, 6, size=n),
        'comment': [fake.text(max_nb_chars=200) for _ in range(n)],
        'review_date': [fake.date_between(start_date='-1y', end_date='today').strftime('%Y-%m-%d') for _ in range(n)]
    }
    return pd.DataFrame(data)

# Generate dummy data for Transactions
def create_transactions(freelancers_df, clients_df, n):
    data = {
        'transaction_id': range(1, n + 1),
        'freelancer_id': np.random.choice(freelancers_df['freelancer_id'], size=n),
        'client_id': np.random.choice(clients_df['client_id'], size=n),
        'amount': [round(np.random.uniform(50, 5000), 2) for _ in range(n)],
        'transaction_date': [fake.date_between(start_date='-1y', end_date='today').strftime('%Y-%m-%d') for _ in range(n)],
        'payment_method': np.random.choice(['credit_card', 'bank_transfer', 'paypal'], size=n)
    }
    return pd.DataFrame(data)

# Generate datasets
num_records = 50
freelancers_df = create_freelancers(num_records)
clients_df = create_clients(num_records)
proposals_df = create_proposals(freelancers_df, clients_df, num_records)
reviews_df = create_reviews(freelancers_df, clients_df, num_records)
transactions_df = create_transactions(freelancers_df, clients_df, num_records)

# SQL Statements
def create_table_sql():
    return """
-- Create tables
CREATE TABLE freelancers (
    freelancer_id INT AUTO_INCREMENT PRIMARY KEY,
    name VARCHAR(100) NOT NULL,
    email VARCHAR(100) UNIQUE NOT NULL,
    phone VARCHAR(15),
    skills TEXT,
    rating FLOAT CHECK (rating >= 0 AND rating <= 5),
    hourly_rate FLOAT,
    profile_description TEXT
);

CREATE TABLE clients (
    client_id INT AUTO_INCREMENT PRIMARY KEY,
    name VARCHAR(100) NOT NULL,
    email VARCHAR(100) UNIQUE NOT NULL,
    phone VARCHAR(15),
    company VARCHAR(100),
    profile_description TEXT
);

CREATE TABLE proposals (
    proposal_id INT AUTO_INCREMENT PRIMARY KEY,
    freelancer_id INT,
    client_id INT,
    project_title VARCHAR(100) NOT NULL,
    description TEXT,
    bid_amount FLOAT,
    submission_date DATE,
    status ENUM('submitted', 'accepted', 'rejected') NOT NULL,
    FOREIGN KEY (freelancer_id) REFERENCES freelancers(freelancer_id),
    FOREIGN KEY (client_id) REFERENCES clients(client_id)
);

CREATE TABLE reviews (
    review_id INT AUTO_INCREMENT PRIMARY KEY,
    freelancer_id INT,
    client_id INT,
    rating INT CHECK (rating >= 1 AND rating <= 5),
    comment TEXT,
    review_date DATE,
    FOREIGN KEY (freelancer_id) REFERENCES freelancers(freelancer_id),
    FOREIGN KEY (client_id) REFERENCES clients(client_id)
);

CREATE TABLE transactions (
    transaction_id INT AUTO_INCREMENT PRIMARY KEY,
    freelancer_id INT,
    client_id INT,
    amount FLOAT,
    transaction_date DATE,
    payment_method ENUM('credit_card', 'bank_transfer', 'paypal'),
    FOREIGN KEY (freelancer_id) REFERENCES freelancers(freelancer_id),
    FOREIGN KEY (client_id) REFERENCES clients(client_id)
);
"""

def generate_sql_insert(df, table_name):
    columns = ', '.join(df.columns)
    values = ', '.join([f"({', '.join([repr(x) for x in row])})" for row in df.values])
    insert_statement = f"INSERT INTO {table_name} ({columns}) VALUES {values};"
    return insert_statement

# Write SQL statements to a file
with open('freelance_marketplace_data.sql', 'w') as f:
    f.write(create_table_sql() + '\n')
    f.write(generate_sql_insert(freelancers_df, 'freelancers') + '\n')
    f.write(generate_sql_insert(clients_df, 'clients') + '\n')
    f.write(generate_sql_insert(proposals_df, 'proposals') + '\n')
    f.write(generate_sql_insert(reviews_df, 'reviews') + '\n')
    f.write(generate_sql_insert(transactions_df, 'transactions') + '\n')

print("SQL file generated successfully.")


SQL file generated successfully.
